<a href="https://colab.research.google.com/github/sbjglitch/Sanskrit2mal/blob/main/sanskrit_to_malayalam_NMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf sample_data
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
%cd /content
!git clone https://github.com/sbjglitch/Sanskrit2mal.git

/content
Cloning into 'Sanskrit2mal'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 35 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (35/35), done.


In [ ]:
%cd /content/Sanskrit2mal/parallel-corpus

mal_lines = []
sanskrit_lines = []
with open ('bhagavadgita_malayalam.txt','r') as f:
  mal_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open ('bhagvadgita_sanskrit.txt','r') as f:
  sanskrit_lines.extend([x.replace('\n', '') for x in f.readlines()])

  print(len(mal_lines))
  print(len(sanskrit_lines))

/content/Sanskrit2mal/parallel-corpus
646
701


In [ ]:
import random
c = list(zip(sanskrit_lines, mal_lines))
random.shuffle(c)

sanskrit_lines, mal_lines = zip(*c)

train_text_sa = sanskrit_lines[:-1374]
train_text_mal = mal_lines[:-374]

dev_text_sa = sanskrit_lines[-1374:]
dev_text_mal = mal_lines[-374:]

In [ ]:
print(len(train_text_mal))
print(len(dev_text_mal))

272
374


In [ ]:
# Start and End tokens
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1